<a href="https://colab.research.google.com/github/Sangram-1409/AgriSmart/blob/main/Continuous_Crypto_Miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python script serves as a simple, continuous crypto miner.
# It is designed to work with a simplified blockchain (like the one we built previously).

# The miner's sole job is to find a valid hash for a new block by
# continuously incrementing the 'nonce' until the hash meets a specific difficulty requirement.

# This script can be run in parallel with a blockchain script that generates transactions.

import hashlib
import json
import time
import os
import threading
import sys

# The target difficulty for the mining puzzle.
# A higher number of leading zeros means more difficult mining.
DIFFICULTY = 4

# This class represents the data that a miner would receive from a network.
# In a real network, this data would be broadcast to all miners.
class MiningJob:
    """A data structure representing the information needed to mine a block."""
    def __init__(self, index, transactions, previous_hash):
        self.index = index
        self.transactions = transactions
        self.previous_hash = previous_hash

    def __str__(self):
        """Provides a string representation of the mining job."""
        return (f"Mining Job for Block #{self.index}\n"
                f"Previous Hash: {self.previous_hash[:15]}...\n"
                f"Transactions: {self.transactions}\n")

# A simple miner that continuously tries to find a valid hash.
class Miner:
    """A miner that performs Proof-of-Work to find a valid block hash."""
    def __init__(self, job, difficulty):
        """Initializes the miner with a job and difficulty level."""
        self.job = job
        self.difficulty = difficulty
        self.nonce = 0
        self.hash = ""

    def calculate_hash(self):
        """Calculates the SHA-256 hash for the current job and nonce."""
        block_string = json.dumps({
            "index": self.job.index,
            "transactions": self.job.transactions,
            "timestamp": time.time(), # The timestamp is set at the time of mining
            "previous_hash": self.job.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()

    def mine(self):
        """
        Performs the Proof-of-Work by trying different nonces.
        Returns the valid hash and the nonce used to find it.
        """
        target = "0" * self.difficulty
        start_time = time.time()

        print("Miner started. Looking for a hash with", self.difficulty, "leading zeros...")

        while not self.hash.startswith(target):
            self.nonce += 1
            self.hash = self.calculate_hash()
            # Provides live feedback on the mining process
            sys.stdout.write(f"\rMining... nonce: {self.nonce:,}, hash: {self.hash[:20]}...")
            sys.stdout.flush()

        end_time = time.time()
        print(f"\n✅ Found valid hash after {self.nonce:,} attempts in {end_time - start_time:.2f} seconds!")
        print(f"Hash: {self.hash}")

        return self.hash, self.nonce

# --- Main demonstration script ---
if __name__ == "__main__":
    # In a real network, the mining job would be received from the blockchain.
    # We will simulate a job here for demonstration purposes.

    # This represents a job to mine Block #1, with a previous hash from a "genesis block"
    # and a set of transactions.
    dummy_transactions = [
        {"from": "Bob", "to": "Alice", "amount": 10},
        {"from": "Charlie", "to": "Bob", "amount": 5}
    ]

    # This is a sample previous hash from a "genesis block"
    previous_hash = "0000216b5a34f4d1c4f5b5f6e8c7c9c0d1b3e8a719d3f4b6a8a3c5d6e7f8g9h0"

    # Create the mining job
    mining_job = MiningJob(index=1, transactions=dummy_transactions, previous_hash=previous_hash)

    # Initialize and run the miner
    miner = Miner(job=mining_job, difficulty=DIFFICULTY)

    try:
        mined_hash, mined_nonce = miner.mine()
        print("\n--- Mining Completed ---")
        print(f"Block Index: {mining_job.index}")
        print(f"Transactions: {mining_job.transactions}")
        print(f"Nonce: {mined_nonce}")
        print(f"Final Hash: {mined_hash}")
        print("--------------------------")
    except KeyboardInterrupt:
        print("\nMiner stopped by user.")

Miner started. Looking for a hash with 4 leading zeros...
Mining... nonce: 31,574, hash: c5c8540eb952309bdfc1...